In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer, InterclusterDistance
from yellowbrick.contrib.scatter import ScatterVisualizer
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabasz_score, adjusted_rand_score
from matplotlib.ticker import FixedLocator, FixedFormatter
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics.cluster import adjusted_rand_score

In [3]:
data =pd.read_csv('https://raw.githubusercontent.com/bachir151/Projet-openclassrooms/master/P4/table_segmentation.csv')
data.head()

,customer_unique_id,Nb_inactive_days,frequency,montant,note_com,frais_livraison,temps_moyen_livraison,Alimentation et Boissons,Auto et Moto,Autres,Livres et Médias,Maison et Jardin,Mode,Santé et Beauté,Sports et Loisirs,Électronique
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0,0,0,0,0,1,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0,0,0,0,0,0,0,1,0,0
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0,0,0,0,0,0,0,0,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62,4.0,17.63,20.0,0,0,0,0,0,0,0,0,1
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89,5.0,16.89,13.0,0,0,0,0,0,0,0,0,1


In [4]:
data.shape

(92493, 16)

In [5]:
segment_1 = data[['customer_unique_id','Nb_inactive_days','frequency','montant']]  # Correspond à table RFM
segment_2 = data[['customer_unique_id','Nb_inactive_days','frequency','montant','note_com','frais_livraison','temps_moyen_livraison']]
segment_3 = data

In [6]:
segment_1.head().rename(columns={'Nb_inactive_days':'recency'})

,customer_unique_id,recency,frequency,montant
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19
2,0000f46a3911fa3c0805444483337064,536,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89


In [7]:
segment_2.head().rename(columns={'Nb_inactive_days':'recency'})

,customer_unique_id,recency,frequency,montant,note_com,frais_livraison,temps_moyen_livraison
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62,4.0,17.63,20.0
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89,5.0,16.89,13.0


# Explication de l'algorithme DBSCAN 

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) est un algorithme de clustering non paramétrique qui peut identifier des groupes de points denses dans un ensemble de données. L'un des avantages de DBSCAN est qu'il peut identifier des clusters de formes arbitraires et peut être efficace pour les ensembles de données contenant du bruit ou des valeurs aberrantes.

L'algorithme DBSCAN fonctionne en définissant deux paramètres clés : epsilon (ε) et minPts. Epsilon est la distance maximale entre deux points pour qu'ils soient considérés comme faisant partie du même cluster, tandis que minPts est le nombre minimum de points requis pour former un cluster. Ces deux paramètres définissent ensemble la densité requise pour qu'un groupe de points soit considéré comme un cluster.

L'algorithme DBSCAN fonctionne comme suit :

- Sélectionnez un point au hasard qui n'a pas encore été visité.

- Trouvez tous les points dans un rayon epsilon autour du point sélectionné.

- Si le nombre de points dans ce rayon est supérieur ou égal à minPts, créez un nouveau cluster et ajoutez tous les points trouvés dans ce rayon.

- Répétez les étapes 2 et 3 pour tous les points dans le nouveau cluster.

- Si le nombre de points dans le rayon est inférieur à minPts, le point est marqué comme du bruit et n'appartient à aucun cluster.

- Répétez les étapes 1 à 5 jusqu'à ce que tous les points aient été visités.

À la fin de l'algorithme, certains points peuvent rester marqués comme du bruit, ce qui signifie qu'ils ne sont pas suffisamment proches d'autres points pour former un cluster.

L'un des avantages de DBSCAN est qu'il peut identifier des clusters de formes arbitraires, contrairement à d'autres algorithmes de clustering tels que k-means qui supposent que les clusters sont sphériques. De plus, DBSCAN peut gérer des ensembles de données contenant du bruit ou des valeurs aberrantes car il ne les inclura pas dans un cluster à moins qu'ils ne répondent aux critères de densité définis par les paramètres epsilon et minPts.

Cependant, il peut être difficile de choisir les bons paramètres pour DBSCAN, car ils dépendent de la densité de l'ensemble de données. Si epsilon est trop grand, des clusters différents peuvent être fusionnés ensemble, tandis que si epsilon est trop petit, des clusters différents peuvent être considérés comme séparés. De même, si minPts est trop petit, de petits groupes de points peuvent être considérés comme des clusters, tandis que si minPts est trop grand, certains points peuvent être considérés comme du bruit même s'ils font partie d'un cluster.

En résumé, DBSCAN est un algorithme de clustering non paramétrique efficace pour l'identification de clusters de formes arbitraires dans les ensembles de données denses. Cependant, le choix des paramètres peut être difficile et dépend de la densité de l'ensemble de données.

# DBSCAN pour ma table segment_1

In [8]:
import warnings

In [9]:
# StandardScaler
scaler = StandardScaler()

segment_1_copy=segment_1.drop(['customer_unique_id'], axis=1).copy()
segment_1_sample_x = segment_1_copy.sample(frac = .30, random_state = 42) # faire le cluster avec l'ensemble des données
segment_1_scaled = scaler.fit_transform (segment_1_sample_x)

Finding Best hyperparameters for DBSCAN using Silhouette Coefficient

In [ ]:
"""%%time
#source : https://thinkingneuron.com/how-to-create-clusters-using-dbscan-in-python/
## Finding best values of eps and min_samples
warnings.simplefilter("ignore")
# Defining the list of hyperparameters to try
eps_list=np.arange(start=0.3, stop=2, step=0.1)
min_sample_list=np.arange(start=25, stop=100, step=25)

# Creating empty data frame to store the silhouette scores for each trials
silhouette_scores_data=pd.DataFrame()

for eps_trial in eps_list:
    for min_sample_trial in min_sample_list:
        
        # Generating DBSAN clusters
        db1 = DBSCAN(eps=eps_trial, min_samples=min_sample_trial)
        
        if len(np.unique(db1.fit_predict(segment_1_scaled ))) > 1:
            sil_score=silhouette_score(segment_1_scaled , db1.fit_predict(segment_1_scaled))
        else:
            continue
        trial_parameters="eps:" + str(eps_trial.round(1)) +" min_sample :" + str(min_sample_trial)
        
        silhouette_scores_data=silhouette_scores_data.append(pd.DataFrame(data=[[sil_score,trial_parameters]], columns=["score", "parameters"]))

# Finding out the best hyperparameters with highest Score
silhouette_scores_data.sort_values(by='score', ascending=False).head(1)"""

Find optimal parameter setting for DBSCAN

In [10]:
%%time
#source : https://www.kaggle.com/code/pnarerdoan/k-means-dbscan-clustering/notebook
epsilon = np.arange(start=0.5, stop=2, step=0.1)
min_samples = np.arange(start=25, stop=100, step=75)


sil_avg = []
tab_performance = []

for i in range(len(epsilon)):
    for j in range(len(min_samples)):

        db2 = DBSCAN(min_samples = min_samples[j], eps =epsilon[i]).fit(segment_1_scaled)
        #cluster_labels=dbscan.fit_predict(data) 
        core_samples_mask = np.zeros_like(db2.labels_, dtype=bool)
        core_samples_mask[db2.core_sample_indices_] = True
        labels = db2.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)


        silhouette_avg = silhouette_score(segment_1_scaled , labels)
        tab_performance.append({'eps': epsilon[i], 'min sample' :min_samples[j], 'nb cluster': n_clusters_, 'silouhette score' :silhouette_avg, 'n noise':n_noise_})
        #sil_avg.append(silhouette_avg)

CPU times: user 4min, sys: 1min, total: 5min
Wall time: 4min 29s


paramètre pour le dbscan

In [11]:
tab_perf=pd.DataFrame(tab_performance)
tab_perf[tab_perf['nb cluster']>=3].sort_values(by='silouhette score', ascending=False).head(10)

,eps,min sample,nb cluster,silouhette score,n noise
7,1.2,25,6,0.353342,215
6,1.1,25,5,0.352738,269
5,1.0,25,5,0.352160,294
4,0.9,25,5,0.350818,329
3,0.8,25,4,0.350558,392
2,0.7,25,4,0.349934,433
1,0.6,25,4,0.348324,482
0,0.5,25,5,0.290796,571


Clustering en utilisant les meilleurs hyperparamètres

In [9]:
segment_1_copy=segment_1.drop(['customer_unique_id'], axis=1).copy()
segment_1_sample_x = segment_1_copy.sample(frac = .30, random_state = 42) # faire le cluster avec l'ensemble des données
segment_1_scaled = scaler.fit_transform (segment_1_sample_x)

In [10]:
# DBSCAN Clustering
from sklearn.cluster import DBSCAN
db3 = DBSCAN(eps=1.2, min_samples=25,n_jobs=-1).fit(segment_1_scaled)
labels = db3.labels_
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

In [11]:
print(f"Silhouette Coefficient: {silhouette_score(segment_1_scaled, labels):.3f}")

Silhouette Coefficient: 0.353


In [12]:
segment_1_sample_x['cluster'] = labels
segment_1_sample_x.groupby('cluster').agg({'Nb_inactive_days' : 'mean',
                                   'frequency' : 'mean',
                                   'montant' : 'mean',
                                   'cluster' :'count'
                                  }).rename(columns={'Nb_inactive_days':'recency'})

,recency,frequency,montant,cluster
cluster,,,,
-1,255.265116,5.948837,3795.356047,215
0,236.951863,1.000000,150.220922,23558
1,236.759296,2.000000,327.385641,3012
2,237.578755,4.000000,688.720916,273
3,234.619211,3.000000,560.698868,583
4,190.986301,5.000000,882.657945,73
5,262.617647,6.000000,585.751176,34


En résumé, le label de cluster -1 dans DBSCAN indique les points qui ne font pas partie d'un cluster et sont considérés comme du bruit ou des valeurs aberrantes

#<ins> DBSCAN pour ma table segment_2 </ins>

In [10]:
# StandardScaler
scaler = StandardScaler()

segment_2_copy=segment_2.drop(['customer_unique_id'], axis=1).copy()
segment_2_sample_x = segment_2_copy.sample(frac = .3, random_state = 42) # faire le cluster avec l'ensemble des données
segment_2_scaled = scaler.fit_transform (segment_2_sample_x)

Finding Best hyperparameters for DBSCAN using Silhouette Coefficient

In [11]:
%%time
#source : https://www.kaggle.com/code/pnarerdoan/k-means-dbscan-clustering/notebook
epsilon = np.arange(start=0.5, stop=2, step=0.1)
min_samples = np.arange(start=25, stop=100, step=75)


sil_avg = []
tab_performance = []

for i in range(len(epsilon)):
    for j in range(len(min_samples)):

        db2 = DBSCAN(min_samples = min_samples[j], eps =epsilon[i]).fit(segment_2_scaled)
        #cluster_labels=dbscan.fit_predict(data) 
        core_samples_mask = np.zeros_like(db2.labels_, dtype=bool)
        core_samples_mask[db2.core_sample_indices_] = True
        labels = db2.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)


        silhouette_avg = silhouette_score(segment_2_scaled , labels)
        tab_performance.append({'eps': epsilon[i], 'min sample' :min_samples[j], 'nb cluster': n_clusters_, 'silouhette score' :silhouette_avg, 'n noise':n_noise_})
        #sil_avg.append(silhouette_avg)

CPU times: user 5min 19s, sys: 49 s, total: 6min 8s
Wall time: 5min 36s


Paramètres pour le dbscan

In [12]:
tab_perf=pd.DataFrame(tab_performance)
tab_perf[tab_perf['nb cluster']>=3].sort_values(by='silouhette score', ascending=False).head(10)

,eps,min sample,nb cluster,silouhette score,n noise
7,1.2,25,4,0.137127,867
6,1.1,25,4,0.135392,1020
3,0.8,25,3,0.117826,2227
5,1.0,25,5,0.115156,1272
4,0.9,25,6,0.105448,1647
2,0.7,25,7,0.049069,2845
0,0.5,25,8,0.031685,4967
1,0.6,25,9,0.030140,3671


Clustering en utilisant les meilleurs hyperparamètres

In [11]:
%%time
# DBSCAN Clustering
from sklearn.cluster import DBSCAN
db3 = DBSCAN(eps=1.2, min_samples=25).fit(segment_2_scaled)
labels = db3.labels_
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

silhouette_avg1 = silhouette_score(segment_2_scaled , labels)
silhouette_avg1

CPU times: user 30.1 s, sys: 6.36 s, total: 36.5 s
Wall time: 37.8 s


0.7187928572194934

In [12]:
print(f"Silhouette Coefficient: {silhouette_score(segment_2_scaled, labels):.3f}")

Silhouette Coefficient: 0.719


In [13]:
segment_2_sample_x['cluster'] = labels
segment_2_sample_x.groupby('cluster').agg({'Nb_inactive_days' : 'mean',
                                   'frequency' : 'mean',
                                   'montant' : 'mean',
                                   'note_com' :'mean',
                                   'frais_livraison' :'mean',
                                   'temps_moyen_livraison':'mean',
                                   'cluster' :'count'
                                  }).rename(columns={'Nb_inactive_days':'recency'})


,recency,frequency,montant,note_com,frais_livraison,temps_moyen_livraison,cluster
cluster,,,,,,,
-1,251.407317,3.951220,2504.946317,3.222207,61.587703,23.019512,410
0,236.723572,1.194345,179.721319,4.164764,19.542002,11.805838,27338
